# 🤖 Task B: Model Attribution Training - SemEval 2026 Task 13

**Goal:** Train models to identify which AI model generated the code (11 classes)

**Level:** ⭐⭐⭐ Advanced (2-3 hours)

**What you'll learn:**
- Multi-class classification (11 classes)
- Model attribution problem
- Handling class imbalance
- Advanced evaluation metrics

**Task B Details:**
- **Classes:** 11 (10 AI models + Human)
- **Expected Baseline:** 35-45% F1
- **Target Performance:** 85%+ F1


In [2]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

from src.features import extract_features_from_dataframe

# Set seed for reproducibility
SEED = 42
np.random.seed(SEED)
import random
random.seed(SEED)

print("✅ Libraries loaded!")
print(f"🔒 Random seed: {SEED}")


✅ Libraries loaded!
🔒 Random seed: 42


## 2. Load Task B Data


In [3]:
# Load Task B datasets
try:
    train_df = pd.read_parquet('task_b_trial.parquet')
    val_df = pd.read_parquet('../data/validation_B.parquet')
    print(f"✅ Data loaded successfully!")
except FileNotFoundError:
    print("❌ Task B data not found!")
    print("📥 Please download Task B data first:")
    print("   python3 src/generate_data.py --task B")
    raise

print(f"\nTraining: {len(train_df):,} samples")
print(f"Validation: {len(val_df):,} samples")
print(f"\nNumber of classes: {train_df['label'].nunique()}")
print(f"Label range: {train_df['label'].min()} to {train_df['label'].max()}")

# Show label distribution
print("\n📊 Label Distribution (Training):")
label_counts = train_df['label'].value_counts().sort_index()
print(label_counts)


❌ Task B data not found!
📥 Please download Task B data first:
   python3 src/generate_data.py --task B


FileNotFoundError: [Errno 2] No such file or directory: '../data/train_B.parquet'

## 3. Visualize Class Distribution


In [ ]:
# Visualize class distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Training distribution
train_counts = train_df['label'].value_counts().sort_index()
ax1.bar(range(len(train_counts)), train_counts.values, color='#42a5f5')
ax1.set_xlabel('Class Label', fontsize=12)
ax1.set_ylabel('Count', fontsize=12)
ax1.set_title('Training Set - Class Distribution', fontsize=14, fontweight='bold')
ax1.set_xticks(range(len(train_counts)))
ax1.set_xticklabels(train_counts.index, rotation=45)
ax1.grid(axis='y', alpha=0.3)

# Validation distribution
val_counts = val_df['label'].value_counts().sort_index()
ax2.bar(range(len(val_counts)), val_counts.values, color='#66bb6a')
ax2.set_xlabel('Class Label', fontsize=12)
ax2.set_ylabel('Count', fontsize=12)
ax2.set_title('Validation Set - Class Distribution', fontsize=14, fontweight='bold')
ax2.set_xticks(range(len(val_counts)))
ax2.set_xticklabels(val_counts.index, rotation=45)
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Class balance check:")
print(f"  Training - Min: {train_counts.min():,}, Max: {train_counts.max():,}, Ratio: {train_counts.max()/train_counts.min():.2f}x")
print(f"  Validation - Min: {val_counts.min():,}, Max: {val_counts.max():,}, Ratio: {val_counts.max()/val_counts.min():.2f}x")


## 4. Extract Features


In [ ]:
# Extract features
print("Extracting features...")
X_train = extract_features_from_dataframe(train_df)
X_val = extract_features_from_dataframe(val_df)
y_train = train_df['label'].values
y_val = val_df['label'].values

print(f"✅ Feature extraction complete!")
print(f"Feature shape: {X_train.shape}")
print(f"Number of features: {X_train.shape[1]}")
print(f"Number of classes: {len(np.unique(y_train))}")
print(f"\nFeature names: {list(X_train.columns)[:10]}...")


## 5. Train Models

**Note:** Task B is harder than Task A (11 classes vs 2). We'll use:
- More trees for Random Forest (200 instead of 100)
- Class balancing to handle potential imbalance
- Multi-class evaluation metrics


In [ ]:
# Define models with adjustments for multi-class problem
models = {
    'Logistic Regression': LogisticRegression(
        max_iter=2000,  # More iterations for 11 classes
        random_state=42,
        class_weight='balanced',  # Handle class imbalance
        multi_class='multinomial'  # Explicit multi-class
    ),
    'Random Forest': RandomForestClassifier(
        n_estimators=200,  # More trees for complex 11-class problem
        max_depth=20,
        min_samples_split=10,
        random_state=42,
        class_weight='balanced',  # Handle class imbalance
        n_jobs=-1
    ),
    'Gradient Boosting': GradientBoostingClassifier(
        n_estimators=200,  # More trees
        max_depth=5,
        learning_rate=0.1,
        random_state=42
    )
}

results = {}

# Train and evaluate each model
for name, model in models.items():
    print(f"\n{'='*60}")
    print(f"Training {name}...")
    print(f"{'='*60}")
    
    # Train
    model.fit(X_train, y_train)
    
    # Predict
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)
    
    # Evaluate
    train_f1 = f1_score(y_train, y_pred_train, average='macro')
    val_f1 = f1_score(y_val, y_pred_val, average='macro')
    
    results[name] = {
        'train_f1': train_f1,
        'val_f1': val_f1,
        'model': model,
        'predictions': y_pred_val
    }
    
    print(f"Train F1: {train_f1:.4f}")
    print(f"Val F1:   {val_f1:.4f}")
    print(f"\nClassification Report (Top 5 classes shown):")
    # Show classification report for all classes
    report = classification_report(y_val, y_pred_val, output_dict=True)
    print(classification_report(y_val, y_pred_val))


In [ ]:
# Create comparison dataframe
comparison_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Train F1': [r['train_f1'] for r in results.values()],
    'Val F1': [r['val_f1'] for r in results.values()]
})

print(comparison_df)

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(comparison_df))
width = 0.35

ax.bar(x - width/2, comparison_df['Train F1'], width, label='Train F1', color='#66bb6a')
ax.bar(x + width/2, comparison_df['Val F1'], width, label='Val F1', color='#42a5f5')

ax.set_xlabel('Model', fontsize=12)
ax.set_ylabel('Macro F1 Score', fontsize=12)
ax.set_title('Task B: Model Comparison (11 Classes)', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(comparison_df['Model'], rotation=15, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)

# Add baseline expectation line
ax.axhline(y=0.40, color='r', linestyle='--', alpha=0.5, label='Expected Baseline (40%)')
ax.axhline(y=0.85, color='g', linestyle='--', alpha=0.5, label='Target (85%)')

plt.tight_layout()
plt.show()


## 7. Confusion Matrix (Best Model)

**Note:** For 11 classes, the confusion matrix will be 11x11. This shows which models are confused with each other.


In [ ]:
# Get best model
best_model_name = max(results, key=lambda k: results[k]['val_f1'])
best_predictions = results[best_model_name]['predictions']

print(f"Best Model: {best_model_name}")
print(f"Validation F1: {results[best_model_name]['val_f1']:.4f}")

# Confusion matrix
cm = confusion_matrix(y_val, best_predictions)

# Create labels for 11 classes
class_labels = [f'Class {i}' for i in range(11)]
# If you know the actual model names, replace above with:
# class_labels = ['Human', 'GPT-3.5', 'GPT-4', 'Claude', 'Codex', ...]

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_labels,
            yticklabels=class_labels,
            cbar_kws={'label': 'Count'})
plt.title(f'Confusion Matrix - {best_model_name} (Task B)', fontsize=14, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Print per-class accuracy
print("\n📊 Per-Class Accuracy:")
for i in range(11):
    class_mask = y_val == i
    if class_mask.sum() > 0:
        class_acc = (best_predictions[class_mask] == i).mean()
        print(f"  Class {i}: {class_acc:.4f} ({class_mask.sum()} samples)")


## 8. Feature Importance (Random Forest)


In [ ]:
# Get feature importance from Random Forest
rf_model = results['Random Forest']['model']
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

# Plot top 15 features
plt.figure(figsize=(10, 8))
top_features = feature_importance.head(15)
plt.barh(range(len(top_features)), top_features['importance'], color='#66bb6a')
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Importance', fontsize=12)
plt.title('Top 15 Most Important Features (Task B)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\nTop 10 Features:")
print(feature_importance.head(10))


## 9. Your Turn! 🎯

**Experiments to try for Task B:**
1. **Hyperparameter tuning** - More trees, different depths
2. **Feature engineering** - Model-specific features (different AI models may have distinct patterns)
3. **Class balancing** - Try different class_weight strategies
4. **Advanced models** - XGBoost, LightGBM often perform better on multi-class
5. **Ensemble methods** - Combine multiple models
6. **AST features** - Structural patterns may differ between AI models

**Add your code below:**


In [ ]:
# Your experiments here!
# Example: Try XGBoost
# from xgboost import XGBClassifier
# xgb_model = XGBClassifier(n_estimators=200, random_state=42)
# xgb_model.fit(X_train, y_train)
# xgb_pred = xgb_model.predict(X_val)
# xgb_f1 = f1_score(y_val, xgb_pred, average='macro')
# print(f"XGBoost F1: {xgb_f1:.4f}")


## 10. Key Takeaways

**Expected baseline performance (Task B):**
- Logistic Regression: ~30-40% F1
- Random Forest: ~35-45% F1
- Gradient Boosting: ~35-45% F1

**Why Task B is harder:**
- 11 classes vs 2 classes (more complex decision boundaries)
- Different AI models may have similar patterns
- Requires more sophisticated features to distinguish models

**To reach competitive performance (85%+ F1):**
- Add AST features (structural patterns differ between models)
- Use transformer models like CodeBERT
- Model-specific feature engineering
- Ensemble multiple models
- Hyperparameter optimization

**Your observations:**
- 
- 
- 


---

## ✅ Next Steps

1. **Analyze confusion matrix** - Which models are confused with each other?
2. **Add AST features** - Structural patterns may help distinguish AI models
3. **Try advanced models** - XGBoost, LightGBM for better multi-class performance
4. **Feature engineering** - Create model-specific features
5. **Hyperparameter tuning** - Optimize for 11-class problem

**Great work on training Task B models!** 🎉
